# Rating Product & Sorting Reviews in Amazon

## 1. Business Problem

One of the key challenges in e-commerce is the accurate calculation of post-purchase ratings for products. Solving this issue not only ensures greater customer satisfaction for e-commerce platforms but also boosts product visibility for sellers and provides a seamless shopping experience for buyers. 

Another significant problem is the accurate sorting of product reviews. As misleading reviews can directly impact product sales, they may result in both financial losses and customer churn. 

Addressing these two fundamental issues will not only increase sales for e-commerce platforms and sellers but also enable customers to complete their purchasing journey smoothly.

## 2. About Dataset

This dataset containing Amazon product data includes various metadata along with product categories. It includes user ratings and reviews for the product with the most reviews in the electronics category.

* **reviewerID:** User ID
* **asin:** Product ID
* **reviewerName:** User Name
* **helpful:** Helpful rating
* **reviewText:** Review
* **overall:** Product rating
* **summary:** Review summary
* **unixReviewTime:** Review time
* **reviewTime:** Review time (Raw)
* **day_diff:** Number of days since the review
* **helpful_yes:** Number of helpful votes for the review
* **total_vote:** Total number of votes for the review

## 3. Data Preparing & Understanding

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import math
from sklearn.preprocessing import MinMaxScaler
import datetime as dt

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
df_ = pd.read_csv('/kaggle/input/amazon-review-csv/amazon_review.csv') #upload the dataset
df = df_.copy() # copy the dataset to protect original dataset

In [3]:
df.head() #display first 5 columns

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [4]:
df.shape #display the shape of the dataset

(4915, 12)

In [5]:
df["overall"].value_counts() #counting 

overall
5.00000    3922
4.00000     527
1.00000     244
3.00000     142
2.00000      80
Name: count, dtype: int64

In [6]:
df.info() #info about dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [7]:
# Converting date columns to datetime type

date_list = ["unixReviewTime", "reviewTime"]

for i in date_list:
    df[i] = pd.to_datetime(df[i])

In [8]:
df.info() #check the datatype again

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   reviewerID      4915 non-null   object        
 1   asin            4915 non-null   object        
 2   reviewerName    4914 non-null   object        
 3   helpful         4915 non-null   object        
 4   reviewText      4914 non-null   object        
 5   overall         4915 non-null   float64       
 6   summary         4915 non-null   object        
 7   unixReviewTime  4915 non-null   datetime64[ns]
 8   reviewTime      4915 non-null   datetime64[ns]
 9   day_diff        4915 non-null   int64         
 10  helpful_yes     4915 non-null   int64         
 11  total_vote      4915 non-null   int64         
dtypes: datetime64[ns](2), float64(1), int64(3), object(6)
memory usage: 460.9+ KB


In [9]:
# we defined 2 days after the last order date as the analysis date

df["reviewTime"].max()
today_date = pd.to_datetime('2014-12-09 00:00:00')

In [10]:
# the difference in days between the analysis date and the comment date

df["days"] = (today_date - df["reviewTime"]).dt.days

In [11]:
df["days"].describe() #descriptive statistics

count   4915.00000
mean     438.36704
std      209.43987
min        2.00000
25%      282.00000
50%      432.00000
75%      602.00000
max     1065.00000
Name: days, dtype: float64

## 4. Rating Products

### 4.1. Calculating "Average Rating" 

In [12]:
#average rating:

df["overall"].mean()

4.587589013224822

### 4.2. Calculating "Time-based Weighted Average Rating"

In [13]:
# defining Time-based Weighted Average function:

def time_based_weighted_average(dataframe, w1=28, w2=26, w3=24, w4=22):
    return dataframe.loc[dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.25), "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.25)) & (dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.50)), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.50)) & (dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.75)), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.75)), "overall"].mean() * w4 / 100

In [14]:
#calculating Time-based Weighted Average:

time_based_weighted_average(df)

4.595593165128118

### 4.3. Comparing the Average of Each Time Period

In [15]:
df.loc[df["day_diff"] <= df["day_diff"].quantile(0.25), "overall"].mean() 

4.6957928802588995

In [16]:
df.loc[(df["day_diff"] > df["day_diff"].quantile(0.25)) & (df["day_diff"] <= df["day_diff"].quantile(0.50)), "overall"].mean()

4.636140637775961

In [17]:
df.loc[(df["day_diff"] > df["day_diff"].quantile(0.50)) & (df["day_diff"] <= df["day_diff"].quantile(0.75)), "overall"].mean()

4.571661237785016

In [18]:
df.loc[(df["day_diff"] > df["day_diff"].quantile(0.75)), "overall"].mean()

4.4462540716612375

**NOTE:**

> * Improvements may have been made to the product over time. 
> * In the scoring, the score of recent comments seems to have increased.

## 5. Sorting Reviews

### 5.1. Calculating Scores

In [19]:
#creating a new column

df["helpful_no"] = df["total_vote"] - df["helpful_yes"]

**NOTE:**

> * **total_vote :** the total number of up-down votes given to a comment.
> * **up :** helpful.
> * **helpful_no :** not helpful.

In [20]:
# up-down difference score
# defining score_pos_neg_diff function:

def score_pos_neg_diff(up,down):
    return up-down

df["score_pos_neg_diff"] = df.apply(lambda x: score_pos_neg_diff(x["helpful_yes"],x["helpful_no"]),axis=1)


In [21]:
# checking the score_pos_neg_diff sorting:

df.sort_values(by="score_pos_neg_diff",ascending=False).head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1970-01-01 00:00:01.367366400,2013-01-05,702,1952,2020,703,68,1884
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1970-01-01 00:00:01.375660800,2013-05-08,579,1568,1694,580,126,1442
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00000,Top of the class among all (budget-priced) mic...,1970-01-01 00:00:01.348617600,2012-09-26,803,1428,1505,804,77,1351
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.00000,"Warning, read this!",1970-01-01 00:00:01.346544000,2012-02-09,1033,422,495,1034,73,349
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.00000,"Resolving confusion between ""Mobile Ultra"" and...",1970-01-01 00:00:01.350864000,2012-10-22,777,112,139,778,27,85


In [22]:
# average rating score
# defining score_average_rating function:

def score_average_rating (up,down):
    if up+down == 0:
        return 0
    return up/(up+down)

df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"],x["helpful_no"]),axis=1)

In [23]:
# checking the score_average_rating sorting:

df.sort_values(by="score_average_rating",ascending=False).head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff,score_average_rating
4277,A35KXSU6AD1481,B007WTAJTO,S. Q.,"[1, 1]",I have a galaxy note II and after rooting I no...,5.00000,Perfect!!,1970-01-01 00:00:01.355875200,2012-12-19,719,1,1,720,0,1,1.00000
2881,A3VSG5X7GPNNW6,B007WTAJTO,Lou Thomas,"[1, 1]",The Nexus One is listed as supporting a maximu...,5.00000,Nexus One Loves This Card!,1970-01-01 00:00:01.349049600,2012-01-10,1063,1,1,1064,0,1,1.00000
1073,A2ZXEKQ2OBZLEE,B007WTAJTO,C. Sanchez,"[1, 1]",I used it with my Samsung S4 and it works grea...,5.00000,Tons of space for phone,1970-01-01 00:00:01.376352000,2013-08-13,482,1,1,483,0,1,1.00000
445,AIWBDRNBODLEA,B007WTAJTO,"Apache ""Elizabeth""","[1, 1]",This is exactly what I was looking for to upgr...,4.00000,Amazon Great Prices,1970-01-01 00:00:01.387324800,2013-12-18,355,1,1,356,0,1,1.00000
3923,A2PH4RGYVR34L,B007WTAJTO,Rock Your Roots,"[1, 1]","It's a SanDisk, so what more is there to say? ...",5.00000,What more to say?,1970-01-01 00:00:01.388361600,2013-12-30,343,1,1,344,0,1,1.00000


In [24]:
# wilson lower bound score
# defining wilson_lower_bound function:

def wilson_lower_bound(up, down, confidence = 0.95):
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound( x["helpful_yes"], x["helpful_no"]), axis = 1)

In [25]:
# checking the wilson_lower_bound sorting:

df.sort_values(by="wilson_lower_bound",ascending=False).head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1970-01-01 00:00:01.367366400,2013-01-05,702,1952,2020,703,68,1884,0.96634,0.95754
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00000,Top of the class among all (budget-priced) mic...,1970-01-01 00:00:01.348617600,2012-09-26,803,1428,1505,804,77,1351,0.94884,0.93652
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1970-01-01 00:00:01.375660800,2013-05-08,579,1568,1694,580,126,1442,0.92562,0.91214
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.00000,"Warning, read this!",1970-01-01 00:00:01.346544000,2012-02-09,1033,422,495,1034,73,349,0.85253,0.81858
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.00000,Super high capacity!!! Excellent price (on Am...,1970-01-01 00:00:01.394150400,2014-07-03,158,45,49,159,4,41,0.91837,0.80811


### 5.2. Determining 20 Reviews to be Displayed on the Product Detail Page

In [26]:
# 20 reviews identified:

df.sort_values(by= "wilson_lower_bound" , ascending = False).head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1970-01-01 00:00:01.367366400,2013-01-05,702,1952,2020,703,68,1884,0.96634,0.95754
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00000,Top of the class among all (budget-priced) mic...,1970-01-01 00:00:01.348617600,2012-09-26,803,1428,1505,804,77,1351,0.94884,0.93652
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1970-01-01 00:00:01.375660800,2013-05-08,579,1568,1694,580,126,1442,0.92562,0.91214
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.00000,"Warning, read this!",1970-01-01 00:00:01.346544000,2012-02-09,1033,422,495,1034,73,349,0.85253,0.81858
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.00000,Super high capacity!!! Excellent price (on Am...,1970-01-01 00:00:01.394150400,2014-07-03,158,45,49,159,4,41,0.91837,0.80811
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.00000,I own it,1970-01-01 00:00:01.393545600,2014-02-28,283,60,68,284,8,52,0.88235,0.78465
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.00000,"Resolving confusion between ""Mobile Ultra"" and...",1970-01-01 00:00:01.350864000,2012-10-22,777,112,139,778,27,85,0.80576,0.73214
3807,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.00000,"Good buy for the money but wait, I had an issue!",1970-01-01 00:00:01.361923200,2013-02-27,649,22,25,650,3,19,0.88000,0.70044
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.00000,Awesome Card!,1970-01-01 00:00:01.339200000,2012-09-06,823,51,65,824,14,37,0.78462,0.67033
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.00000,Designed incompatibility/Don't support SanDisk,1970-01-01 00:00:01.348272000,2012-09-22,807,82,109,808,27,55,0.75229,0.66359
